**SVM MODEL**

In [54]:
import pandas as pd
import numpy as np


from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

In [55]:
# Loading dataset

In [56]:
data=pd.read_csv('forestfires.csv')
data.tail()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small
516,nov,tue,79.5,3.0,106.7,1.1,11.8,31,4.5,0.0,...,0,0,0,0,0,0,1,0,0,small


**DATA PREPROCESSING**

In [57]:
np.unique(data['size_category'])

array(['large', 'small'], dtype=object)

In [58]:
data.columns

Index(['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind',
       'rain', 'area', 'dayfri', 'daymon', 'daysat', 'daysun', 'daythu',
       'daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 'monthfeb',
       'monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov',
       'monthoct', 'monthsep', 'size_category'],
      dtype='object')

In [59]:
data['size_category'] = data['size_category'].map({'large': 1, 'small': 0})
data


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,0
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,0
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,0
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,0
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,1
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,1
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,1
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
data2=data.drop(['month', 'day'],1)
data2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,1,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,1,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,1
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,1
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,1
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
x=data2.iloc[:,:28]
y=data2.iloc[:,28]

In [62]:
# Splitting data into training and testing data set

In [63]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [64]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((361, 28), (361,), (156, 28), (156,))

**Tuning svm model with different values of parameters 'gmma' , 'c' and kernal='rbf' for best accuracy**

In [65]:
model = SVC()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5,60],'C':[15,14,13,12,11,10,0.1,0.001,1,2] }]
gsv = GridSearchCV(model,param_grid,cv=10)
gsv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [15, 14, 13, 12, 11, 10, 0.1, 0.001, 1, 2],
                          'gamma': [50, 5, 10, 0.5, 60], 'kernel': ['rbf']}])

In [66]:
# Accoriding to above gridsearch method  these are the best parameters 

In [67]:
gsv.best_params_ , gsv.best_score_ 

({'C': 15, 'gamma': 0.5, 'kernel': 'rbf'}, 0.7368618618618619)

In [68]:
# Building model with above parameteres

In [69]:
model_2=SVC(C=15,gamma=0.5,kernel='rbf')
model_2.fit(x_train , y_train)
y_pred = model_2.predict(x_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)

Accuracy = 75.0


**We got the accuracy as 71.1% for our model.Here there is difference of 4% accuracy for train and test data**

**Tuning svm model only 'c' parameter and kernal='rbf'**

In [70]:
# Keeping 'gamma' as defalut

In [71]:
model_test = SVC()
param_grid = [{'kernel':['rbf'],'C':[15,0.1,0.001,1,2,16,17,20,25,30,50,120] }]
gsv = GridSearchCV(model_test,param_grid,cv=10)
gsv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [15, 0.1, 0.001, 1, 2, 16, 17, 20, 25, 30, 50,
                                120],
                          'kernel': ['rbf']}])

In [72]:
# Accoriding to above gridsearch parameters these are the best 

In [73]:
gsv.best_params_ , gsv.best_score_ 

({'C': 120, 'kernel': 'rbf'}, 0.9390390390390391)

**Final Model**

In [74]:
# Building model with above parameteres

In [75]:
model_final = SVC(C=120,kernel='rbf')
model_final.fit(x_train , y_train)

SVC(C=120)

In [76]:
# Predicting for test data

In [77]:
y_pred = model_final.predict(x_test)

In [78]:
confusion_matrix(y_test, y_pred)

array([[115,   1],
       [ 12,  28]])

In [79]:
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)

Accuracy = 91.66666666666666


**We got the accuracy as 91.1% for our model.Here there is only difference of 1% accuracy for train and test data**